<a href="https://colab.research.google.com/github/Jagaskak/CourseProject/blob/main/ITMTF_GoogleColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

nytimes_fp = "/content/drive/My Drive/Data/NYTimes/"
stockprices_fp = "/content/drive/My Drive/Data/PriceHistory/"

Mounted at /content/drive/


In [2]:
!pip install pyLDAvis

import re
import numpy as np
import pandas as pd
from pprint import pprint
import datetime
from collections import defaultdict

# NLTK
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import ngrams

# Gensim
import gensim
from gensim import models
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import Phrases

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt 

from scipy.stats import pearsonr

import statsmodels
from statsmodels.tsa.stattools import grangercausalitytests

import warnings
warnings.filterwarnings('ignore')

     |████████████████████████████████| 1.6MB 5.6MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=d70944d0ebd0a3375e2be23931d6a8c7ba78da8294aee8c5605c6b6acc34a802
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
'''
This cell contains the processing of NYTimes data and IEM stock market data. You should only run this once.
'''

stop_words = stopwords.words('english')
stop_words.extend(['mr', 'ms', 'mrs', 'said'])

# Tokenize and remove stop words from content
def tokenize(content, lemmatize=False):
    words = gensim.utils.simple_preprocess(content, deacc=True)  # tokenizes
    return words

def remove_stopwords(content):
    words = []
    for word in content:
        if word in stop_words:
            continue
        words.append(word)
    return words


'''
Retrieve Data from files
'''

# New York Times Data
rows = []
dates = []
articles = []
for month in range(5, 11):
    with open(nytimes_fp + str(month) + ".txt") as f:
        for i, line in enumerate(f):
            date, article = line.split(",", 1)
            timestamp = datetime.datetime.strptime(date, "%Y-%m-%dT%H:%M:%S%z").date()
            tokenized = tokenize(article)
            destopped = remove_stopwords(tokenized)

            articles.append(destopped)
            dates.append(timestamp)
            rows.append([timestamp, destopped])

nytimes = pd.DataFrame(rows, columns=["Date", "Content"]) 
unique_dates = sorted(list(set(nytimes["Date"])))
# print (unique_dates)

# Time Series Data
ts_months = ["May", "Jun", "Jul", "Aug", "Sep", "Oct"]
cols = ['Date', 'LastPrice']
stock_prices = pd.DataFrame()
for month in ts_months:
    ts_df = pd.read_csv(stockprices_fp + month + ".txt", delim_whitespace=True)
    ts_df['Date'] =  ts_df['Date'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%y").date())
    
    Gore = ts_df.loc[ts_df['Contract'] == 'Dem'][['Date', 'LastPrice']].fillna(0).reset_index()
    Bush = ts_df.loc[ts_df['Contract'] == 'Rep'][['Date', 'LastPrice']].fillna(0).reset_index()

    # Gore/(Gore + Bush)
    relation = list(zip(Gore['Date'], (Gore['LastPrice']/(Gore['LastPrice'] + Bush['LastPrice'])).fillna(0.001)))
    stock_prices = stock_prices.append(relation, ignore_index=True)

stock_prices.columns = cols


'''
BOW - corpus, date x word cnts
'''
# Create Dictionary
id2word = corpora.Dictionary(articles)

# Filtering: 
# Keep words that appear in at least 1% of docs --> approximately 50 topics
# Don't keep words that appear in more that 70% of docs in corpus
# id2word.filter_extremes(no_below=50, no_above=0.7)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in articles]

# TF-IDF seems to give better coherence (but it wasn't in the paper...)
tfidf = models.TfidfModel(corpus)
tfidf_corpus = tfidf[corpus]

# Human readable format of corpus (term-frequency)
# [[(id2word[id], freq) for id, freq in cp] for cp in tfidf_corpus[:1]][0][:5]

# bow by date
date_term_cnts = defaultdict(lambda: [])

for index, row in nytimes.iterrows():
    date = row["Date"]
    content = row["Content"]
    
    date_term_cnts[date] += content
    
date_term_cnts = list(date_term_cnts.items())
date_term_cnts = [(date, {id2word[id]: freq for id, freq in id2word.doc2bow(text)}) for date, text in date_term_cnts]
date_term_cnts = sorted(date_term_cnts, key=lambda x: x[0])
date_term_cnts = pd.DataFrame([date_term_cnts[i][1] for i in range(len(date_term_cnts))], index=[date_term_cnts[i][0] for i in range(len(date_term_cnts))]).fillna(0.0)


In [7]:
'''
This cell contains function definitions.
'''

def rel_purity(currPurity, prevPurity):
    return abs(currPurity - prevPurity)/prevPurity


# Select the model and print the topics
def get_topics(lda_model, num_topics=-1, num_words=100, prob_thresh=0.8):
    topics = []
    for topic, topic_words in lda_model.print_topics(num_topics=num_topics, num_words=num_words):
        words = topic_words.split(" + ")
        all_words = []
        all_prob = 0
        for elem in words:
            prob, word = elem.split("*")
            all_prob += float(prob)
            all_words.append(word.split('"')[1])

            if all_prob >= prob_thresh:
                break
        topics.append((topic, all_words))
    
    return topics


'''
Significant portion of grangers_causality_matrix function was taken from stackoverflow post:
https://stackoverflow.com/questions/58005681/is-it-possible-to-run-a-vector-autoregression-analysis-on-a-large-gdp-data-with
'''
def grangers_causality_matrix(data, variables, maxlag=5, test='ssr_ftest', verbose=False):
    dataset = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    lags    = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    
    for c in dataset.columns:
        for r in dataset.index:            
            test_result = grangercausalitytests(data[[r,c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1], 4) for i in range(maxlag)]
            
            if verbose: 
                print(f'Y = {r}, X = {c}, P Values = {p_values}')

            # smaller p-val corresponds to higher f-val
            min_p_value_i = np.argmin(p_values)
            min_p_value = p_values[min_p_value_i]
            dataset.loc[r, c] = min_p_value
            
            lags.loc[r, c] = min_p_value_i
   
    return dataset, lags

def get_causal_vars(data, significance=0.95, getLags=False, getCausalSig=False, verbose=False):
    cols = data.columns[:-1]
    causal_vars = []
    causal_lags = []
    
    for col in cols:
        try:
            gc, lags = grangers_causality_matrix(data[[col, 'LastPrice']], 
                                             variables=[col, 'LastPrice'], 
                                             verbose=False)
        except:
            raise Exception(data[[col, 'LastPrice']])
        
        gc = 1 - gc
        
        col_causes = gc.loc['LastPrice', col] >= significance
        col_causedBy = gc.loc[col, 'LastPrice'] >= significance
        if col_causes or col_causedBy:
            if getCausalSig:
                causal_vars.append((col, max(gc.loc['LastPrice', col], gc.loc[col, 'LastPrice'])))
            else:
                causal_vars.append(col)
            
            if getLags:
                # if granger causality for topic causing ts and ts causing topic both significant, choose higher sig
                if col_causes and col_causedBy:
                    if gc.loc['LastPrice', col] >= gc.loc[col, 'LastPrice']:
                        causal_lags.append(lags.loc['LastPrice', col])
                    else:
                        causal_lags.append(lags.loc[col, 'LastPrice'] * -1)
                elif col_causes:
                    causal_lags.append(lags.loc['LastPrice', col])
                else:
                    causal_lags.append(lags.loc[col, 'LastPrice'] * -1)
    if getLags:
        return causal_vars, causal_lags
    
    return causal_vars
                
def get_word_stream(nytimes, topics, causal_topics):
    ct_ws = []
    for ct in causal_topics:
        causal_vocab = list(set(topics[ct][1]))
        date_terms = pd.DataFrame(np.zeros((len(unique_dates), len(causal_vocab))), index=unique_dates, columns=causal_vocab)
        
        for word in causal_vocab:
            date_terms[word] = date_term_cnts[word]
            
        ct_ws.append((ct, date_terms))
    
    return ct_ws

def get_impact_words(topic_wordstream, significance=0.95, verbose=False):
    topic_impact_words = []
    
    for topic, ws in topic_wordstream:
        ws_prices = ws.join(stock_prices.set_index('Date')).dropna()        
        ws_gc = get_causal_vars(ws_prices, significance=significance, getCausalSig=True, verbose=verbose)
        
        pos = []
        neg = []
        for word, sig in ws_gc:                
            corr = pearsonr(ws_prices[word], stock_prices['LastPrice'])[0]
            if corr >= 0:
                pos.append((word, sig))
            else:
                neg.append((word, sig))
                
        topic_impact_words.append((topic, pos, neg))
    
    return topic_impact_words
        
def construct_prior(impact_words, curr_k, const_k_increase=0, sig=0.95, alter_k=True):
    # find number of topics that we are splitting
    if alter_k:
        new_k = curr_k + len(impact_words) + const_k_increase
    else:
        new_k = curr_k
    word_priors = np.zeros((new_k, date_term_cnts.shape[1])) + (1/len(id2word))

    i = 0
    for topic_num, pos, neg in impact_words:
        pos_denom = sum([granger-sig for word, granger in pos])
        neg_denom = sum([granger-sig for word, granger in neg])
        
        if len(pos) < 0.1 * len(neg):
            # num neg words >> num pos
            for word, granger in pos:              
                word_priors[i, id2word.token2id[word]] = 0
            for word, granger in neg:
                word_priors[i, id2word.token2id[word]] = (granger-sig)/neg_denom 
            i += 1
            continue
            
        elif len(neg) < 0.1 * len(pos):
            # num pos words >> num neg
            for word, granger in pos:              
                word_priors[i, id2word.token2id[word]] = (granger-sig)/pos_denom 
            for word, granger in neg:
                word_priors[i, id2word.token2id[word]] = 0
            i += 1
            continue
            
        for word, granger in pos:              
            word_priors[i, id2word.token2id[word]] = (granger-sig)/pos_denom 
        
        for word, granger in neg:
            word_priors[i + 1, id2word.token2id[word]] = (granger-sig)/neg_denom 
        
        i += 2
    
    return word_priors
            
def calculate_purity(pWords, nWords):
    n = float(len(pWords) + len(nWords))
    if n == 0:
        return 0
    pProb = len(pWords)/n
    nProb = len(nWords)/n
    
    pProb = pProb if pProb else 1
    nProb = nProb if nProb else 1
    
    entropy = pProb * np.log(pProb) + nProb * np.log(nProb)
    purity = 100 + 100 * entropy
    return purity

def calculate_conf(impact_words):
    all_conf = 0.0
    num_words = 0
    for topic, pos, neg in impact_words:
        num_words += len(pos) + len(neg)
        for words, gc in pos:
            all_conf += gc

        for words, gc in neg:
            all_conf += gc

    return all_conf/num_words*100 if num_words else 0


def show_plot(x, yData, xaxislabel, yaxislabel, labels, xticks=None, yaxisrange=None, title=None, legend_title=None, marker=".", saveAs=None):
    if len(yData) != len(labels):
        raise ValueError("Number of labels should equal number of lines you want to plot")
    
    plt.xlabel(xaxislabel)
    plt.ylabel(yaxislabel)
    plt.grid()
        
    for i in range(len(yData)):
        plt.plot(x, yData[i], label=labels[i], marker=marker)
    plt.legend(loc=2, bbox_to_anchor=(1.05, 1), title=legend_title)
    
    if yaxisrange:
        plt.ylim(yaxisrange)
    if xticks:
        plt.xticks(xticks)
    if title:
        plt.title(title)
    
    if saveAs:
        plt.savefig(saveAs, bbox_inches='tight')
    plt.show()

In [8]:
'''
This function will run topic modeling with time series feedback 

Parameters:
paramCtrl  -- str (either 'k' or 'decay'), tells function to test various k or decay values
params     -- list of integers, each val in list is either a k or decay values to test.
              For k values, the last k value will be used to test a variable number of topics starting at k topics.
              In every iteration of ITMTF, k will increase by the number of causal topics found + some constant
decay      -- (0.5, 1] decay parameter to use when running lda (effective only when paramCtrl='k')
k          -- Interger representing number of topics to use when running lda (effective only when paramCtrl='decay')
iterations       -- Integer number of iterations to run ITMTF
const_k_increase -- Integer number of topics to increase (in addition to number of causal topics found) 
                    each iteration when running varying number of topics (effective only when paramCtrl='k')
                    Default is 0
verbose          -- True/False Print out purity and confidence every iteration

Returns:
k_lda_model    -- The final lda_model  
k_avg_purities -- Average purity of all causal topics in each iteration
k_avg_conf     -- Average confidence of all causal topics in each iteration
'''
def ITMTF(paramCtrl, params, decay=1, k=30, const_k_increase=5, iterations=5, verbose=True):
    thirdLDAModel = None
    
    prevPurity = 100
    purity = 0
    alpha = "auto"
    eta = "auto"
    alter_k = False

    all_avg_purities = []
    all_avg_conf = []

    for val in params:
        if paramCtrl == "k":
            k = val
        elif paramCtrl == "decay":
            decay = val
        else:
            raise ValueError("Invalid argument. Please read docs to understand use case")
        
        if verbose:
            print ("\n\nk: ", k)
            print ("decay: ", decay)

        avg_purities = []
        avg_confidences = []

        num_iter = 0
        while num_iter < iterations:

            if verbose and alter_k == "k":
                print ("\nNumber topics: ", k)          

            lda_model = gensim.models.ldamodel.LdaModel(
                                    corpus=tfidf_corpus,
                                    id2word=id2word,
                                    num_topics=k, 
                                    alpha=alpha,
                                    eta=eta,
                                    decay=decay)

            topics = get_topics(lda_model, prob_thresh=0.3, num_words=50)
            document_topics = lda_model.get_document_topics(corpus)
            date_doc_topics = list(zip(nytimes["Date"], lda_model.get_document_topics(corpus)))

            # for any given day, you look at all the diff topics and identify the prob of that topic
            date_topic_prob = np.zeros((len(unique_dates), k))
            for date, article in date_doc_topics:
                i = unique_dates.index(date)
                for topic, prob in article:
                    date_topic_prob[i][topic] += prob 

            date_topic = pd.DataFrame(date_topic_prob, index=unique_dates)
            date_topic["Date"] = unique_dates

            date_topic_prices = date_topic.set_index('Date').join(stock_prices.set_index('Date')).dropna()
            causal_topics = get_causal_vars(date_topic_prices)
            ct_ws = get_word_stream(nytimes, topics, causal_topics)

            impact_words = get_impact_words(ct_ws)

            # Calculate Purity
            purities = [calculate_purity(topic[1], topic[2]) for topic in impact_words]
            if len(purities) == 0:
                # Try again, we got no causal topics
                continue

            avg_purity = sum(purities)/len(purities)
            prevPurity = purity
            purity = avg_purity

            avg_purities.append(purity)

            # Calculate Confidence
            avg_conf = calculate_conf(impact_words)
            avg_confidences.append(avg_conf)
            
            if verbose:
                print ("Purity: ", purity)  
                print ("Avg. Conf: ", avg_conf)

            # Prior for next iteration
            eta = construct_prior(impact_words, k, const_k_increase=const_k_increase, alter_k=alter_k)

            # adjust num topics
            if alter_k:
                k += len(impact_words) + const_k_increase

    #         if keep3rd and (i + 1) == 3:
    #             thirdLDAModel = lda_model
            num_iter += 1

        all_avg_purities.append(avg_purities)
        all_avg_conf.append(avg_confidences)

        # reset params
        eta = "auto"

        if paramCtrl == "k":
            alter_k = (k == params[-2])

    return lda_model, all_avg_purities, all_avg_conf

k_lda_model, k_avg_purities, k_avg_conf = ITMTF("k", params=[10, 20, 30, 40, 10], const_k_increase=3)
# mu_lda_model, mu_avg_purities, mu_avg_conf = ITMTF("decay", params=[0.51, 0.6, 0.7, 0.8, 0.9, 1])



k:  10
decay:  1
Purity:  76.22590918988769
Avg. Conf:  97.12599999999998
Purity:  45.830909574139845
Avg. Conf:  97.51153846153848
Purity:  43.35827545160637
Avg. Conf:  96.61866666666667
Purity:  52.451818379775375
Avg. Conf:  97.03545454545453
Purity:  77.47193955668477
Avg. Conf:  96.95499999999998


k:  20
decay:  1
Purity:  31.709189529952823
Avg. Conf:  97.75999999999999
Purity:  54.581998589112345
Avg. Conf:  97.43473684210527
Purity:  40.65924807596458
Avg. Conf:  97.86999999999999
Purity:  61.84409152221347
Avg. Conf:  97.3885714285714
Purity:  62.16007013052867
Avg. Conf:  97.61869565217391


k:  30
decay:  1
Purity:  62.80194994145747
Avg. Conf:  98.09214285714285
Purity:  47.819697347843
Avg. Conf:  97.87491525423727
Purity:  43.30216381409163
Avg. Conf:  97.77888888888886
Purity:  56.22351014010727
Avg. Conf:  98.01142857142858
Purity:  43.9130564413375
Avg. Conf:  97.9355172413793


k:  40
decay:  1
Purity:  56.62183857592694
Avg. Conf:  97.80181818181819
Purity:  55.6

In [ ]:
'''
Plotting purity and confidence for different number of topics
'''

k_labels = ["10", "20", "30", "40", "Varying Number of Topics"]
show_plot(range(1, 6), k_avg_purities, 
          "Iteration", "Average Purity",     
          k_labels, 
          xticks=range(1, 6), yaxisrange=[0, 120], 
          title="Average Confidence for Different Number of Topics",
          legend_title="Number of Topics",
          saveAs="purity_k.png")
show_plot(range(1, 6), k_avg_conf,     
          "Iteration", "Average Confidence", 
          k_labels, 
          xticks=range(1, 6), yaxisrange=[95.5, 100], 
          title="Average Confidence for Different Number of Topics",
          legend_title="Number of Topics",
          saveAs="confidence_k.png")

In [ ]:
'''
Plotting purity and confidence for different values of decay
'''

mu_labels = ["0.51", "0.6", "0.7", "0.8", "0.9", "1"]
show_plot(range(1, 6), mu_avg_purities, 
          "Iteration", "Average Purity", 
          mu_labels, 
          xticks=range(1, 6), yaxisrange=[0, 120], 
          title="Average Purity for Different Decay Values", 
          legend_title="Decay Values", 
          saveAs="purity_mu.png")
show_plot(range(1, 6), mu_avg_conf, 
          "Iteration", "Average Confidence", 
          mu_labels, 
          xticks=range(1, 6), yaxisrange=[95.5, 100], 
          title="Average Confidence for Different Decay Values", 
          legend_title="Decay Values", 
          saveAs="confidence_mu.png")

In [ ]:
# Visualize the topic model results lda model (w/ varying number of topics) using pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(k_lda_model, corpus, id2word)
pyLDAvis.save_html(vis, 'LDAvis_k.html')
vis

In [ ]:
# Visualize the topic model results lda model (w/ mu=1 and k=30) using pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(mu_lda_model, corpus, id2word)
pyLDAvis.save_html(vis, 'LDAvis_mu.html')
vis